In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt

# XGBoost modeli için DMatrix formatına dönüştürme
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# XGBoost parametreleri
params = {
    'objective': 'binary:logistic',  # İkili sınıflandırma için
    'eval_metric': 'logloss',       # Değerlendirme metriği
    'eta': 0.1,                     # Öğrenme oranı
    'max_depth': 6,                 # Maksimum derinlik
    'seed': 42                      # Rastgelelik için sabit değer
}

# XGBoost modelini eğitme
num_round = 100  # Eğitim için tur sayısı
bst = xgb.train(params, dtrain, num_round)

# Tahmin yapma
y_pred_prob = bst.predict(dtest)
y_pred = (y_pred_prob > 0.5).astype(int)

# Modelin doğruluğunu hesaplama
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Model Accuracy: {accuracy:.2f}")

# Detaylı rapor (Precision, Recall, F1-Score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Class 0", "Class 1"]))

# Karmaşıklık matrisi oluşturma ve görselleştirme
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix for XGBoost")
plt.show()

# ROC-AUC Hesaplama ve Grafik (İkili sınıflandırma kontrolü)
if len(set(y_test)) == 2:  # Yalnızca ikili sınıflandırma için geçerli
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f"ROC-AUC: {roc_auc:.2f}")
    plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("XGBoost - ROC Curve")
    plt.legend()
    plt.show()
